In [1]:
import requests
import json
import io
import pandas as pd
from datetime import date, timedelta, datetime
import urllib.parse

pd.set_option('display.max_columns', None)

### data dictionary for forestry 
https://docs.google.com/spreadsheets/d/1yMfZgcsrvx9M0b3-ZdEQ3WCk2dFxgitCWytTrJSwEAs/edit#gid=0

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
datetime.now()

datetime.datetime(2022, 10, 7, 14, 47, 8, 269213)

In [4]:
thirty_days_ago = datetime.now() - timedelta(days=30)
thirty_days_ago

datetime.datetime(2022, 9, 7, 14, 47, 8, 461143)

In [5]:
thirty_days_ago.strftime("%Y-%m-%dT%H:%M:%S")

'2022-09-07T14:47:08'

In [6]:
d = urllib.parse.quote_plus(thirty_days_ago.strftime("%Y-%m-%dT%H:%M:%S"))
d

'2022-09-07T14%3A47%3A08'

In [7]:
# # data also available as geojson if we want; different URL & loader
# api_forWO = f"https://data.cityofnewyork.us/resource/bdjm-n7q4.json?$where=createddate>'{d}'"
# # https://dev.socrata.com/foundry/data.cityofnewyork.us/bdjm-n7q4
# api_forWO

In [8]:
# response_forWO = requests.get(api_forWO)
# print(response_forWO.status_code) # 200 means yay

In [9]:
def get_data(url):
    api = requests.get(url)
    print(api.status_code)
    data = api.text
    js = json.loads(data)
    return pd.DataFrame.from_dict(js)

tree_points (~1M rows): https://data.cityofnewyork.us/Environment/Forestry-Tree-Points/hn5i-inap

service_requests (734K rows, created 2017): https://data.cityofnewyork.us/Environment/Forestry-Service-Requests/mu46-p9is

inspections (~1.23M rows): https://data.cityofnewyork.us/Environment/Forestry-Inspections/4pt5-3vv4

work_orders (~943K rows): https://data.cityofnewyork.us/Environment/Forestry-Work-Orders/bdjm-n7q4

---

planting spaces (~1M rows): https://data.cityofnewyork.us/Environment/Forestry-Planting-Spaces/82zj-84is

risk assessments: https://data.cityofnewyork.us/Environment/Forestry-Risk-Assessments/259a-b6s7

tasks (~151K rows): https://data.cityofnewyork.us/Environment/Forestry-Tasks/45sb-srb5

tree contract work details: https://data.cityofnewyork.us/Environment/Tree-Contract-Work-Details/kcfe-uypz


In [41]:
# $limit= 100,000, since 2022

tree_points = get_data('https://data.cityofnewyork.us/resource/hn5i-inap.json?$limit=100000')
service_requests = get_data("https://data.cityofnewyork.us/resource/mu46-p9is.json?$where=createddate>'01/01/2022%2012:00:00'&$limit=100000")
inspections = get_data("https://data.cityofnewyork.us/resource/4pt5-3vv4.json?$where=createddate>'2022-01-01T12:00:00.000'&$limit=100000")
work_orders = get_data('https://data.cityofnewyork.us/resource/bdjm-n7q4.json?$limit=100000')

200
200
200
200


In [42]:
tree_points.shape, service_requests.shape, inspections.shape, work_orders.shape

((100000, 20), (100000, 44), (100000, 25), (100000, 55))

In [25]:
tree_points.head(3)

,objectid,dbh,tpstructure,tpcondition,plantingspaceglobalid,geometry,globalid,genusspecies,createddate,updateddate,location,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,riskrating,riskratingdate,planteddate,stumpdiameter
0,86823,20,Full,Excellent,E814CD37-9F53-4D79-AF86-3B454F9D29B9,POINT (1035099.5043752342 200293.75541685522),2B457A4C-E0E4-4E17-81C4-A5449F51C804,Acer nigrum - black maple,2015-02-28T05:00:00.000,2016-10-20T17:43:53.000,"{'latitude': '40.71629069421649', 'longitude':...",24670,25,3,24,65,NaN,NaN,NaN,NaN
1,87623,10,Retired,Good,A644AB79-A3CB-4F7F-923B-F308E615CCD4,POINT (1001279.3033335656 235478.27354185283),37195E1A-A7EE-4AA4-8389-19A0ED5C46F7,Fraxinus pennsylvanica - Green ash,2015-03-03T05:00:00.000,2019-09-18T13:12:55.000,"{'latitude': '40.81299281873849', 'longitude':...",13095,18,4,36,20,NaN,NaN,NaN,NaN
2,88023,24,Retired,Poor,21431016-EDB8-4A0B-B122-673125800C87,POINT (1030579.8881252259 262707.9770835191),6BA8E72B-1901-4EF3-ABFF-D11680AB4A9B,Acer platanoides - Norway maple,2015-03-03T05:00:00.000,2018-03-27T14:00:42.000,"{'latitude': '40.88762508279444', 'longitude':...",11275,29,5,2,30,NaN,NaN,NaN,NaN


In [26]:
service_requests.head(3)

,objectid,complaintnumber,srcategory,srtype,srpriority,srsource,srstatus,srresolution,boroughcode,communityboard,buildingnumber,streetname,crossstreet1,crossstreet2,taxclass,notestocustomer,zipcode,citycouncil,statesenate,stateassembly,congressional,physicalid,servicerequestparentglobalid,globalid,initiateddate,closeddate,createddate,updateddate,descriptor1,complainttype,callerzipcode,srcallertype,callercity,callerstate,latitude,longitude,census_tract,bin,bbl,nta,locationdetails,complaintdetails,parkname,parkzone
0,17579546,311-02514396,Remove Tree,Dead Tree Older Than 2 Years,Routine,Department of Parks and Recreation - Public We...,Closed,Inspected - Condition Not Found,Brooklyn,310,341,79TH ST,3 AVENUE,4 AVENUE,1,At the time of inspection tree is in fair cond...,11209,43,22,64,11,0037876,20CCF930-5FD1-4B09-9AA3-8576E1744415,FFA207F8-4B0A-46E3-96F8-252A40661027,05/19/2020 14:20:45,05/21/2020 14:19:51,05/19/2020 14:20:00,05/21/2020 14:19:51,Planted More Than 2 Years Ago,Dead/Dying Tree,11209,Business Business,Brooklyn,NY,40.628261,-74.027653,64,3150325,3059690065,Bay Ridge,NaN,NaN,NaN,NaN
1,17616360,311-02553143,Hazard,Limb Down - Street,Routine,3-1-1 Call Center,Closed,Work Completed,Staten Island,501,0,HOWARD AVE,GRYMES HILL ROAD,HOWARD AVENUE,NaN,work order created to remove limb,10301,49,23,61,11,NaN,NaN,6E6381BF-3C40-4B04-AB17-3D676E71F99A,05/23/2020 20:47:06,05/26/2020 13:26:35,05/23/2020 21:12:00,05/26/2020 13:26:35,Branch or Limb Has Fallen Down,Damaged Tree,NaN,Business Business,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEAR HOWARD AVE,LARGE BRANCHES FROM A CITY TREE HAVE FALLEN IN...,NaN,NaN
2,17575577,311-02510143,Hazard,Tree Split,Routine,3-1-1 Call Center,Closed,Inspected - No Work Order,Brooklyn,302,37,CAMBRIDGE PLACE,GREENE AVENUE,GATES AVENUE,NaN,At the time of inspection the tree was found i...,11238,35,25,57,8,0038994,NaN,E6BB2CF4-6559-4ADD-B064-17069BC1B20F,05/18/2020 23:11:51,05/19/2020 15:26:25,05/19/2020 00:00:17,05/19/2020 15:26:25,Tree Trunk Split,Damaged Tree,NaN,NaN,NaN,NaN,40.685902,-73.963283,201,3056193,3019650016,Clinton Hill,NaN,THERE'S A TREE THAT HAS A CRACKED AROUND IT. &...,NaN,NaN


In [50]:
service_requests.srstatus.unique()

array(['Closed', 'Open'], dtype=object)

In [52]:
open_srs = service_requests[service_requests.srstatus=='Open']

In [27]:
inspections.head(3)

,objectid,inspectiontype,inspectionstatus,inspectiontpcondition,inspectiontpstructure,treepointdbh,treepointglobalid,plantingspaceglobalid,geometry,globalid,inspectiondate,closeddate,createddate,updateddate,location,servicerequestglobalid
0,9446994,Level 2,Closed,Good,Full,19,F7A24A87-9E02-452B-9C64-4A83789B4CC3,54091277-1878-48A0-85FC-C0628687C82F,POINT (1052789.5427085608 189089.14770852029),FB2EF480-67DE-4C7F-96C6-06F5D5996F45,2019-12-30T05:00:00.000,2020-01-01T18:47:00.000,2020-01-01T18:46:00.000,2020-01-01T18:47:00.000,"{'latitude': '40.685417428985794', 'longitude'...",NaN
1,9447394,Level 2,Closed,Dead,Shaft,7,2E7CBC93-EBEA-48DC-ABD6-52068828099B,DBCE4626-866D-4B43-AEE3-8DE4ACFA0A2E,POINT (1041397.1783335656 183906.71583351493),525E7B48-2210-40FD-AB8D-01D9799E1728,2019-12-30T05:00:00.000,2020-01-01T18:54:00.000,2020-01-01T18:51:00.000,2020-01-23T13:24:47.000,"{'latitude': '40.671273751636626', 'longitude'...",13FBD50D-402B-4E87-ADD6-70540A06997B
2,9447395,Level 2,Closed,Dead,Shaft,7,2E7CBC93-EBEA-48DC-ABD6-52068828099B,DBCE4626-866D-4B43-AEE3-8DE4ACFA0A2E,POINT (1041397.1783335656 183906.71583351493),5EA08451-7D21-4A05-8B58-6DE59BA6C21B,2019-12-30T05:00:00.000,2020-01-01T19:09:00.000,2020-01-01T19:07:00.000,2020-01-23T13:24:56.000,"{'latitude': '40.671273751636626', 'longitude'...",13FBD50D-402B-4E87-ADD6-70540A06997B


In [28]:
work_orders.head(3)

,objectid,wotype,wostatus,boroughcode,communityboard,buildingnumber,streetname,wocategory,wowoodremains,zipcode,citycouncil,statesenate,stateassembly,congressional,physicalid,crossstreet1,crossstreet2,inspectionglobalid,globalid,wowireconflict,geometry,createddate,woentity,woproject,latitude,longitude,census_tract,bin,bbl,nta,location,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,locationdetails,actualfinishdate,wocontract,closeddate,updateddate,projstartdate,recommendedspecies,crewglobalid,parkname,parkzone,wopriority,sidewalkdamage,cancelreason,canceldate,woequipment,sanitationassigneddate,plantingspaceglobalid,treepointglobalid,sanitationupdateddate
0,10828374,Tree and Sidewalk Repair,Pending,Bronx,202,1050,PROSPECT AVENUE,Tree and Sidewalk Repair,No Wood Debris,10459,17,32,79,15,36384,EAST 166 STREET,EAST 165 STREET,34AC0C34-BC24-41D5-B134-554F116CC91B,544A40B1-6DE4-4597-A59F-2DF76D28E3FD,No Util Conflict Present,POINT (1012112.967916891 239822.27562518418),2019-12-17T22:06:00.000,Capital,"T&S - 5,500 Commitments",40.824542,-73.899589,131,2005243,2026910015,Longwood,"{'latitude': '40.82488834805127', 'longitude':...",10937,8,5,43,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2870515,Tree Plant-Street Tree,Closed,Manhattan,108,1011,LEXINGTON AVENUE,Tree Planting,No Wood Debris,10021,4,28,73,12,3385,EAST 73 STREET,EAST 72 STREET,B86B461B-5DB5-4EB4-96C0-5FCDFE76E891,9B05E01A-83B4-4B87-A179-5B6091AA8AF6,No Util Conflict Present,POINT (994826.457083568 220094.79729184508),2016-02-25T02:37:00.000,Capital,NaN,40.770821,-73.961908,128,1042858,1014070051,Upper East Side-Carnegie Hill,"{'latitude': '40.77077940999285', 'longitude':...",12083,23,4,51,11,1 F EPW/R 51X33/NPW/R 84X48 SR 8-12' REMOVE DE...,04/20/2016 04:00:00,MG-214M,2020-02-03T20:05:00.000,2020-02-03T20:05:00.000,2016-03-15T00:00:00.000,Gymnocladus dioicus - Kentucky coffeetree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8376155,Misc Work,Closed,Manhattan,107,200,WEST 109 STREET,Misc Work,No Wood Debris,10025,7,31,69,10,164225,BROADWAY,AMSTERDAM AVENUE,E707A7CD-A38F-4850-87B5-0FAE179895DB,9F137A96-2646-4C41-87B8-DC6FCCC42503,No Util Conflict Present,POINT (994012.08875022829 231647.84145851433),2018-06-14T15:28:00.000,In-House,NaN,40.802395,-73.964375,195,1075380,1018807501,Morningside Heights,"{'latitude': '40.80249036187423', 'longitude':...",12422,20,4,23,15,NaN,02/21/2020 05:00:00,NaN,2020-02-24T15:33:00.000,2020-02-24T15:33:00.000,NaN,NaN,4C41BC66-1766-4D70-9017-2B57C097CAA7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
tree_points.columns

Index(['objectid', 'dbh', 'tpstructure', 'tpcondition',
       'plantingspaceglobalid', 'geometry', 'globalid', 'genusspecies',
       'createddate', 'updateddate', 'location', ':@computed_region_efsh_h5xi',
       ':@computed_region_f5dn_yrer', ':@computed_region_yeji_bk3q',
       ':@computed_region_92fq_4b7q', ':@computed_region_sbqj_enih',
       'riskrating', 'riskratingdate', 'planteddate', 'stumpdiameter'],
      dtype='object')

In [30]:
service_requests.columns

Index(['objectid', 'complaintnumber', 'srcategory', 'srtype', 'srpriority',
       'srsource', 'srstatus', 'srresolution', 'boroughcode', 'communityboard',
       'buildingnumber', 'streetname', 'crossstreet1', 'crossstreet2',
       'taxclass', 'notestocustomer', 'zipcode', 'citycouncil', 'statesenate',
       'stateassembly', 'congressional', 'physicalid',
       'servicerequestparentglobalid', 'globalid', 'initiateddate',
       'closeddate', 'createddate', 'updateddate', 'descriptor1',
       'complainttype', 'callerzipcode', 'srcallertype', 'callercity',
       'callerstate', 'latitude', 'longitude', 'census_tract', 'bin', 'bbl',
       'nta', 'locationdetails', 'complaintdetails', 'parkname', 'parkzone'],
      dtype='object')

In [31]:
inspections.columns

Index(['objectid', 'inspectiontype', 'inspectionstatus',
       'inspectiontpcondition', 'inspectiontpstructure', 'treepointdbh',
       'treepointglobalid', 'plantingspaceglobalid', 'geometry', 'globalid',
       'inspectiondate', 'closeddate', 'createddate', 'updateddate',
       'location', 'servicerequestglobalid'],
      dtype='object')

In [32]:
work_orders.columns

Index(['objectid', 'wotype', 'wostatus', 'boroughcode', 'communityboard',
       'buildingnumber', 'streetname', 'wocategory', 'wowoodremains',
       'zipcode', 'citycouncil', 'statesenate', 'stateassembly',
       'congressional', 'physicalid', 'crossstreet1', 'crossstreet2',
       'inspectionglobalid', 'globalid', 'wowireconflict', 'geometry',
       'createddate', 'woentity', 'woproject', 'latitude', 'longitude',
       'census_tract', 'bin', 'bbl', 'nta', 'location',
       ':@computed_region_efsh_h5xi', ':@computed_region_f5dn_yrer',
       ':@computed_region_yeji_bk3q', ':@computed_region_92fq_4b7q',
       ':@computed_region_sbqj_enih', 'locationdetails', 'actualfinishdate',
       'wocontract', 'closeddate', 'updateddate', 'projstartdate',
       'recommendedspecies', 'crewglobalid', 'parkname', 'parkzone',
       'wopriority', 'sidewalkdamage', 'cancelreason', 'canceldate',
       'woequipment', 'sanitationassigneddate', 'plantingspaceglobalid',
       'treepointglobalid', '

## possible joins
- plantingspaceglobalid: tree points, work orders, planting spaces
    - tree points plantingspaceglobalid Can be joined to Global ID from Forestry Planting Spaces dataset
    - work orders plantingspaceglobalid Can be joined to Global ID from Forestry Planting Spaces dataset
    - inspections plantingspaceglobalid Can be joined to Global ID from Forestry Planting Spaces dataset
- treepointglobalid: work orders, inspections, tree points
    - work orders treepointglobalid Can be joined to GlobalID from Forestry Tree Points dataset
    - inspections treepointglobalid Can be joined to GlobalID from Forestry Tree Points dataset
- servicerequestglobalid: inspections and service requests!
    - inspections servicerequestglobalid Can be joined to Global ID from Forestry Service Requests dataset
- inspectionglobalid: riskassessments, work orders, inspections
    - riskassessments inspectionglobalid Can be joined to Global ID from Forestry Inspections
    - work orders inspectionglobalid Can be joined to GlobalID from Forestry Inspections dataset
- workorderglobalid: riskassessments, work orders
    - riskassessments workorderglobalid Can be joined to Global ID from Forestry Work Orders
    
and there is also the tasks dataset: https://data.cityofnewyork.us/Environment/Forestry-Tasks/45sb-srb5

In [ ]:
# we would expect a bunch of open things reported waiting their first inspection, 
# then some open things that have been inspected and not worked on, 
# and then a bunch of worked on things that are closed

# what I would do is a left join where inspection is on the left. 
# and then left join in the work orders, where the SR/inspection df is on the left

In [33]:
inspections['servicerequestglobalid']

0                                       NaN
1      13FBD50D-402B-4E87-ADD6-70540A06997B
2      13FBD50D-402B-4E87-ADD6-70540A06997B
3      289D4654-CE56-461B-BBBF-278ECDDBCAE8
4                                       NaN
                       ...                 
995                                     NaN
996                                     NaN
997                                     NaN
998    AAC05B74-1F72-4A0B-8D0C-6E1D08EF09D0
999    250DA51B-BACF-4070-AF06-97B7316A7F75
Name: servicerequestglobalid, Length: 1000, dtype: object

In [53]:
SRinspect = pd.merge(inspections, open_srs, left_on='servicerequestglobalid', right_on='globalid')
SRinspect.head()

,objectid_x,inspectiontype,inspectionstatus,inspectiontpcondition,inspectiontpstructure,treepointdbh,treepointglobalid,plantingspaceglobalid,geometry,globalid_x,inspectiondate,closeddate_x,createddate_x,location,updateddate_x,parentinspectionglobalid,reinspectiondate,servicerequestglobalid,swtreeconditionrating,swlocationrating,swclearancerating,swdamagerating,swdamagedflagstonesrating,swratingtotal,swtotaltrees,objectid_y,complaintnumber,srcategory,srtype,srpriority,srsource,srstatus,srresolution,boroughcode,communityboard,buildingnumber,streetname,crossstreet1,crossstreet2,taxclass,notestocustomer,zipcode,citycouncil,statesenate,stateassembly,congressional,physicalid,servicerequestparentglobalid,globalid_y,initiateddate,closeddate_y,createddate_y,updateddate_y,descriptor1,complainttype,callerzipcode,srcallertype,callercity,callerstate,latitude,longitude,census_tract,bin,bbl,nta,locationdetails,complaintdetails,parkname,parkzone
0,15406340,Sidewalk Rating,Closed,Good,Full,11,1F812A90-24A0-42D1-AA11-5AE3D6BE92A4,9FB5DB51-8B14-4152-ABE9-7A58D8C69D4A,POINT (915367.06354188919 122496.59166684747),728ED54F-D2D3-4EE0-8B22-5A0574F41F87,2022-01-04T14:32:16.000,2022-01-04T14:32:16.000,2022-01-04T14:32:16.000,"{'latitude': '40.502632394006774', 'longitude'...",NaN,NaN,NaN,C1046D8F-DE2B-4C44-B329-C525BFBB8885,4,1,4,5,5,74,2,22327272,311-08303386,Root/Sewer/Sidewalk,Trees and Sidewalks,Routine,3-1-1 Call Center,Open,NaN,Staten Island,503,218,ASPINWALL STREET,HYLAN BOULEVARD,CLERMONT AVENUE,NaN,NaN,10307,51,24,62,11,0081014,NaN,C1046D8F-DE2B-4C44-B329-C525BFBB8885,11/05/2021 20:51:44,NaN,11/05/2021 20:55:10,NaN,Trees and Sidewalks Program,Root/Sewer/Sidewalk Condition,NaN,NaN,NaN,NaN,40.502489,-74.247571,24402,5114201,5079240020,Charleston-Richmond Valley-Tottenville,NaN,The tree roots are damaging sidewalk and the d...,NaN,NaN
1,15411003,Sidewalk Rating,Closed,Good,Full,10,78CB7268-F913-4F09-98C4-B88249DD3464,1BB38283-E9A9-43AE-96EB-7743D4D43101,POINT (914296.18479189277 124661.80750018358),380895F6-5FB6-479C-B099-F722C1C17D23,2022-01-04T16:28:44.000,2022-01-04T16:28:44.000,2022-01-04T16:28:44.000,"{'latitude': '40.50856711920287', 'longitude':...",NaN,NaN,NaN,117F4E35-481F-4265-88F8-3420691E1BB9,4,1,4,5,5,66,1,22539194,311-08554994,Root/Sewer/Sidewalk,Trees and Sidewalks,Routine,Department of Parks and Recreation - Public We...,Open,NaN,Staten Island,503,572,CRAIG AVENUE,PATTEN STREET,BEND,1,NaN,10307,51,24,62,11,0015249,NaN,117F4E35-481F-4265-88F8-3420691E1BB9,11/29/2021 15:54:03,NaN,11/29/2021 15:54:04,NaN,Trees and Sidewalks Program,Root/Sewer/Sidewalk Condition,NaN,NaN,Staten Island,NY,40.508516,-74.251524,248,5089339,5080500058,Charleston-Richmond Valley-Tottenville,NaN,() The city owned tree and roots directly in f...,NaN,NaN
2,15423072,Level 2,Closed,Poor,Full,30,5AFA7BEB-F4CA-4D72-B39C-5BA45F19A6D7,08A1F385-D788-470B-A75E-0E2C14ED13DF,POINT (1044848.5375002325 221071.83208352327),E723F4EE-41FF-4446-8BE5-F22DB10B37CD,2022-01-05T16:51:55.000,2022-01-05T16:51:55.000,2022-01-05T16:51:55.000,"{'latitude': '40.77325972956185', 'longitude':...",NaN,NaN,NaN,E1CD8B9A-6D76-4403-A250-2646AF518A46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22342134,311-08327874,Remove Tree,Dead Tree Older Than 2 Years,Routine,3-1-1 Call Center,Open,Work Priority Category - C,Queens,411,208-42,30 AVENUE,208 STREET,CORPORAL KENNEDY STREET,NaN,Inspected 1/5/2022. Work order created for tre...,11360,19,11,26,6,0096314,NaN,E1CD8B9A-6D76-4403-A250-2646AF518A46,11/08/2021 03:56:13,NaN,11/08/2021 04:00:00,02/24/2022 13:20:12,Planted More Than 2 Years Ago,Dead/Dying Tree,11360,Business Business,Queens,NY,40.773138,-73.781534,1093,4134537,4060420022,Bayside-Bayside Hills,In front of house between sidewalk and street,In front of my house is a very old tree. The ...,NaN,NaN
3,15423604,Level 2,Closed,Good,Full,40,10D6CB1D-EE2D-42B7-B89D-4EDF719C4E66,8C15EEA0-8B92-4041-87AF-F369C7341CD4,POINT (1033319.121041894 228353.35270851851),E77EC3B1-4BB0-4368-BA2F-8C23CCA7AC01,2022-01-05T19:00:4

In [45]:
SRinspect.shape

(1974, 69)

In [46]:
service_requests.globalid

0        FFA207F8-4B0A-46E3-96F8-252A40661027
1        6E6381BF-3C40-4B04-AB17-3D676E71F99A
2        E6BB2CF4-6559-4ADD-B064-17069BC1B20F
3        5DD596B0-B9BF-4A8B-B167-1C80B96B7AC4
4        6C5FFB7E-14FB-4D2C-8B2D-4FE2A4A474AE
                         ...                 
99995    8D768F91-7A66-4443-909B-671C4695DD53
99996    22AF3B23-99ED-4840-BCA8-1D809B12431B
99997    87A2B72D-9F3E-42ED-BD5E-252F68CAB8C5
99998    8F6CB585-2FCD-4158-B72B-82B410E4266B
99999    18F13060-7CAA-4A11-880D-87D0B5AD7E45
Name: globalid, Length: 100000, dtype: object

In [47]:
service_requests.objectid

0        17579546
1        17616360
2        17575577
3        17251858
4        17651915
           ...   
99995    10735969
99996     4703564
99997     4683180
99998    17436630
99999    13178059
Name: objectid, Length: 100000, dtype: object

In [48]:
inspections.globalid

0        988A50F1-89C3-4352-9079-AD86403654B5
1        103A2CC3-9853-4E43-883A-800E64CD4517
2        17C37841-EF21-40EF-9BAA-F175F40E2B4A
3        2CC30002-113C-4AA4-B217-CFE8CEC9EAB9
4        DE62D9F5-27C0-49A4-B8B3-89F53B48B040
                         ...                 
99995    F54BDCDC-3ABF-42B0-8F3E-E478C36C2C59
99996    056D3E20-70C1-44A4-B34D-7897EE20AC92
99997    AA0451EE-1CE4-4953-B1A1-26248C7AA58A
99998    5E3493FA-7F95-4B8D-843D-6160537CA6A4
99999    B3A63B5B-CB2D-4A80-A24A-488A678B768E
Name: globalid, Length: 100000, dtype: object

In [49]:
inspections.objectid

0        15396740
1        15396738
2        15396744
3        15396741
4        15396742
           ...   
99995    17490399
99996    17489055
99997    17489769
99998    17502919
99999    17489761
Name: objectid, Length: 100000, dtype: object